In [ ]:
# Copyright 2025 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

### State 처리 분석

In [20]:
import os
import time
from dotenv import load_dotenv
import argparse

from google.genai import types
from google.adk.runners import Runner
from google.adk.events import Event, EventActions

from google.adk.sessions import BaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import VertexAiSessionService

from agent import root_agent

In [3]:
# 환경 변수 로드.
from dotenv import dotenv_values
config = dotenv_values("../../.env")
for key, value in config.items():
    print(f"{key}: {value}")

GOOGLE_GENAI_USE_VERTEXAI: TRUE
GOOGLE_CLOUD_PROJECT: ai-hangsik
GOOGLE_CLOUD_LOCATION: us-central1
GOOGLE_GENAI_MODEL: gemini-2.5-flash
GOOGLE_GENAI_LIVE_MODEL: gemini-live-2.5-flash
AGENT_ENGINE_BUCKET: gs://agent_engine_0120
AGENTOPS_API_KEY: 43150e85-1c93-4c65-8526-3ac50cf1bcfc


In [13]:
APP_NAME = "AI_assistant"
USER_ID = "Forusone"

session_service = InMemorySessionService()


In [14]:
# 초기 상태 정의
init_state = {
    "task_status": "아직 답변 없음",
    "timestamp": time.time(),
}

In [15]:
session = await session_service.create_session(
    app_name= APP_NAME,
    user_id=USER_ID,
    state=init_state
)
print(f"\n\n1. 초기 세션 상태: {session.state}")




1. 초기 세션 상태: {'task_status': '아직 답변 없음', 'timestamp': 1768995656.23591}


In [17]:
runner = Runner(agent=root_agent,
                app_name=APP_NAME,
                session_service=session_service)

In [18]:
print(f"AI Agent를 종료하려면 'exit' 또는 'quit'을 입력하세요.")

while True:
    print("\n👤 User :   ")

    # 사용자 입력에서 쿼리 가져오기
    query = input("")
    if query.strip().lower() in ["exit", "quit"]:
        break
    else:
        print(query)
    
    content = types.Content(role='user', parts=[types.Part(text=query)])

    # 에이전트 러너를 사용하여 에이전트 이벤트를 비동기적으로 실행합니다.
    # 세션 정보는 user_id와 session_id를 통해 전달됩니다.
    events = runner.run_async(user_id=session.user_id,
                            session_id=session.id,
                            new_message=content,)

    async for event in events:
        if event.is_final_response():
            final_response = event.content.parts[0].text            
            print("\n 🤖 AI Assistant: " + final_response)


    updated_session = await session_service.get_session(app_name = session.app_name, 
                                                     user_id = session.user_id, 
                                                     session_id = session.id)

    # 업데이트된 세션 상태에서 'last_turn' 표시
    print(f"\n\n output_key - last_turn : {updated_session.state['last_turn']}")


AI Agent를 종료하려면 'exit' 또는 'quit'을 입력하세요.

👤 User :   
생성형 AI 란 ?

 🤖 AI Assistant: 생성형 AI(Generative AI)는 사용자의 요청에 따라 텍스트, 이미지, 비디오, 오디오 또는 소프트웨어 코드와 같은 새롭고 독창적인 콘텐츠를 생성할 수 있는 인공지능(AI)의 한 분야입니다. 단순히 기존 데이터를 분석하는 것을 넘어, 학습된 데이터를 기반으로 새로운 결과물을 창조하는 능력을 가졌다는 점에서 기존 AI와 차별화됩니다.

**주요 특징 및 작동 방식:**
*   **새로운 콘텐츠 생성:** 생성형 AI는 사용자의 프롬프트(명령)에 따라 독창적인 콘텐츠를 만들어냅니다.
*   **학습 기반:** 대규모 데이터 세트로부터 패턴, 구조, 관계를 학습하여 새로운 데이터를 생성합니다. 이 과정에서 딥러닝(Deep Learning), 신경망(Neural Networks), 머신러닝(Machine Learning)과 같은 고급 기술이 활용됩니다.
*   **주요 모델:** 변이형 오토인코더(VAE), 생성적 적대 신경망(GAN), 트랜스포머(Transformer) 등이 생성형 AI 모델의 핵심 아키텍처입니다. 특히 트랜스포머는 대규모 언어 모델(LLM)의 기반이 됩니다.

**활용 분야:**
생성형 AI는 예술, 디자인, 문학, 음악, 소프트웨어 개발, 마케팅 등 다양한 분야에서 활용됩니다. 예를 들어, 특정 스타일의 예술 작품을 만들거나, 소설을 쓰고, 코드를 작성하며, 새로운 제품 디자인을 제안하는 등 인간의 창의성을 모방하거나 확장하는 데 기여합니다.

**기존 AI와의 차이점:**
기존의 AI는 주로 정해진 규칙이나 알고리즘에 따라 특정 작업을 수행하거나 데이터를 분석하는 데 중점을 둡니다. 반면, 생성형 AI는 학습된 데이터를 바탕으로 새로운 데이터 인스턴스를 생성할 수 있다는 점에서 차이가 있습니다.


 output_key - last_turn : 생성형 AI(Generative AI)는 사용

In [21]:
# 상태 변경 준비
state_changes = {
    "task_status": f"답변 완료 : {query}",
    "timestamp": time.time(),
    "last_turn": updated_session.state['last_turn']
}

system_event = Event(
    invocation_id = "change-state",
    author = "system", # 또는 'agent', 'tool' 등
    actions = EventActions(state_delta=state_changes),
    timestamp = time.time()
)

# 상태 변경 이벤트 추가
await session_service.append_event(session, system_event)

print("\n\n2. 새로운 명시적 state-delta 이벤트를 추가했습니다.")



2. 새로운 명시적 state-delta 이벤트를 추가했습니다.


In [22]:

updated_session = await session_service.get_session(app_name=session.app_name,
                                            user_id=session.user_id, 
                                            session_id=session.id)

print(f"\n\n3. 이벤트를 보낸 후 세션 상태: {updated_session.state}")





3. 이벤트를 보낸 후 세션 상태: {'task_status': '답변 완료 : exit', 'timestamp': 1768995751.978991, 'last_turn': '생성형 AI(Generative AI)는 사용자의 요청에 따라 텍스트, 이미지, 비디오, 오디오 또는 소프트웨어 코드와 같은 새롭고 독창적인 콘텐츠를 생성할 수 있는 인공지능(AI)의 한 분야입니다. 단순히 기존 데이터를 분석하는 것을 넘어, 학습된 데이터를 기반으로 새로운 결과물을 창조하는 능력을 가졌다는 점에서 기존 AI와 차별화됩니다.\n\n**주요 특징 및 작동 방식:**\n*   **새로운 콘텐츠 생성:** 생성형 AI는 사용자의 프롬프트(명령)에 따라 독창적인 콘텐츠를 만들어냅니다.\n*   **학습 기반:** 대규모 데이터 세트로부터 패턴, 구조, 관계를 학습하여 새로운 데이터를 생성합니다. 이 과정에서 딥러닝(Deep Learning), 신경망(Neural Networks), 머신러닝(Machine Learning)과 같은 고급 기술이 활용됩니다.\n*   **주요 모델:** 변이형 오토인코더(VAE), 생성적 적대 신경망(GAN), 트랜스포머(Transformer) 등이 생성형 AI 모델의 핵심 아키텍처입니다. 특히 트랜스포머는 대규모 언어 모델(LLM)의 기반이 됩니다.\n\n**활용 분야:**\n생성형 AI는 예술, 디자인, 문학, 음악, 소프트웨어 개발, 마케팅 등 다양한 분야에서 활용됩니다. 예를 들어, 특정 스타일의 예술 작품을 만들거나, 소설을 쓰고, 코드를 작성하며, 새로운 제품 디자인을 제안하는 등 인간의 창의성을 모방하거나 확장하는 데 기여합니다.\n\n**기존 AI와의 차이점:**\n기존의 AI는 주로 정해진 규칙이나 알고리즘에 따라 특정 작업을 수행하거나 데이터를 분석하는 데 중점을 둡니다. 반면, 생성형 AI는 학습된 데이터를 바탕으로 새로운 데이터 인스턴스를 생성할 수 있다는 점에서 차이가 있습니다.'}
